#### Created a new Notebook for this assignment and importing pandas to import & manipulate the data. 

In [1]:
import pandas as pd

#### Code to scrape the Wikipedia page,https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M and set to dataframe for further processing


In [2]:
data = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
df=data[0]

#### Printing the dataframe to check the data

In [3]:
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


#### Printing the shape of the dataframe/data

In [4]:
df.shape

(180, 3)

#### Processing the rows that have an assigned borough. Ignore cells with a borough that is 'Not assigned'

In [5]:
df_Borough_NA = df[df ['Borough']== 'Not assigned'].index
df_refined1 = df.drop(df_Borough_NA , inplace=False)
df_refined1.reset_index(drop = True, inplace=True)
df_refined1.head()

,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


#### Printing the shape of dataframe after dropping the rows with Boroughs value as Not assigned

In [6]:
df_refined1.shape

(103, 3)

#### If a cell has a borough but a Not assigned neighborhood, then assiging the neighborhood value to the borough.

In [7]:
df_refined1.loc[(df_refined1.Neighbourhood == 'Not assigned'),'Neighbourhood']='df_refined1[Borough]'


#### Printing the shape of the dataframe

In [8]:
df_refined1.shape


(103, 3)

#### Installing the geocoder library for getting the longtudes and latitudes of the postal codes

In [9]:
! pip install geocoder
import geocoder
# import geocoder

#### Created a function to get the latitudes and Longitudes by passing the postal codes to the function 

In [ ]:
def get_geocoder(postal_code_from_df):
     # initialize your variable to None
     lat_lng_coords = None
     # loop until you get the coordinates
     while(lat_lng_coords is None):
       g = geocoder.google('{}, Toronto, Ontario'.format(postal_code_from_df))
       lat_lng_coords = g.latlng
     latitude = lat_lng_coords[0]
     longitude = lat_lng_coords[1]
     return latitude,longitude



for i in range (0,df_refined1.shape[0]):
    df_refined1['Latitude'][i],df_refined1['Longitude'][i]=get_geocoder(df_refined1.iloc[i]['Postal Code'])


In [ ]:
df_refined1.shape[0]
df_refined1.head()


 #### Geocoder function was not returning the data, so reading the data from the CSV file for retrieving the longitudes & latitudes

In [7]:
df_latlongdata = pd.read_csv("http://cocl.us/Geospatial_data",error_bad_lines=False)
df_latlongdata.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [8]:
df_latlongdata.shape

(103, 3)

#### Merging the two dataframes based on postal code to include latitude and longitude for the locations

In [9]:
df_refined2=pd.merge(df_refined1, df_latlongdata, on="Postal Code")
df_refined2.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [10]:
df_refined2.shape

(103, 5)

### Define Foursquare Credentials and version

In [11]:
CLIENT_ID = 'CJCSVCSCK5P1DQ5CLTDA3PERYFG0OZRLHJF4T142PS4JF4CJ' # your Foursquare ID
CLIENT_SECRET = 'FQGVRV25WWFLRFCF4CZMEGTX5BODQ55LWLK0E5OZZE1VIJ3J' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: CJCSVCSCK5P1DQ5CLTDA3PERYFG0OZRLHJF4T142PS4JF4CJ
CLIENT_SECRET:FQGVRV25WWFLRFCF4CZMEGTX5BODQ55LWLK0E5OZZE1VIJ3J


In [30]:
df_refined3 = df_refined2[df_refined2['Borough'].str.contains('Toronto', case = False)].reset_index(drop=True)

In [31]:
df_refined3.head()


,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


In [34]:
!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

In [35]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto,Ontario are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto,Ontario are 43.6534817, -79.3839347.


In [36]:
!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    altair-4.1.0               |             py_1         614 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


altair-4.1.0         | 614 KB    | ##################################### | 1

In [57]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(df_refined3['Latitude'], df_refined3['Longitude'], df_refined3['Borough']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [ ]:
import json
! pip install requests
import requests # library to handle requests
from pandas.io.json import json_normalize

In [60]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [64]:
toronto_venues = getNearbyVenues(names=df_refined3['Borough'],
                                   latitudes=df_refined3['Latitude'],
                                   longitudes=df_refined3['Longitude'])

Downtown Toronto


NameError: name 'requests' is not defined

In [53]:
print(toronto_venues.shape)
toronto_venues.head()

NameError: name 'toronto_venues' is not defined

In [ ]:
toronto_venues.groupby('Neighborhood').count()

In [ ]:
print('There are {} uniques categories.'.format(len(manhattan_venues['Venue Category'].unique())))